**Identificação do aluno**

**Email:** lucas.ariel.carvalho@ccc.ufcg.edu.br

**Matrícula:** 12210801

# Laboratório: DistilBERT com Máscara de Linguagem (IMDB)

Diferente dos laboratórios anteriores, onde a maioria das seções já estava pronta, **neste laboratório você será o protagonista da construção**.  
Você deverá se basear no exemplo visto em aula e **desenvolver sua própria versão**, colocando sua **personalidade** e aplicando **toques extras** para buscar um melhor desempenho.  

Relembrando o laboratório exemplificado em aula:  
- Ele utiliza o modelo **DistilBERT** em uma tarefa de **Masked Language Modeling (MLM)**.  
- O objetivo é treinar o modelo em um conjunto de dados (IMDB) e observar sua capacidade de prever palavras mascaradas em frases.  
- O processo envolve:  
  1. Preparação dos dados.  
  2. Tokenização.  
  3. Criação de máscaras nas frases.  
  4. Fine-tuning do modelo.  
  5. Avaliação dos resultados.  

Agora é a sua vez!  
Use o que foi mostrado em aula como **guia**, mas explore variações. Use essas **DICAS**:  
- Experimente **diferentes taxas de aprendizado**.  
- Aplique **diferentes tamanhos de batch**.  
- Teste **número de épocas distintos**.
- Use **diferentes taxas de mascaramento**.   
- Pense em **formas criativas de avaliar o desempenho**.  

O objetivo não é apenas replicar o exemplo, mas **melhorá-lo com suas próprias ideias**.

# Bibliotecas

In [22]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from transformers import DistilBertTokenizer, TFDistilBertForMaskedLM # DICA: Tente usar outras versões de BERT

# Conjunto de Dados

In [23]:
# Esse conjunto de dados é relacionado ao IMDB (Plataforma que contempla resenhas sobre filmes)
dataset = load_dataset('imdb')
print(dataset['train'][0]['text'])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [24]:
from datasets import DatasetDict
from sklearn.model_selection import train_test_split

def stratified_subset_dataset(dataset_dict, n_samples_per_split, seed=42):
    """
    Recebe um DatasetDict (por exemplo, {'train': ..., 'test': ...})
    e retorna outro DatasetDict com a mesma estrutura,
    mas com no máximo n_samples_per_split exemplos em cada split,
    mantendo a proporção das labels.
    """
    new_splits = {}

    for split_name, split_data in dataset_dict.items():
        labels = np.array(split_data['label'])
        indices = np.arange(len(labels))
        # Seleção estratificada
        selected_indices, _ = train_test_split(
            indices,
            train_size=min(n_samples_per_split, len(labels)),
            stratify=labels,
            random_state=seed
        )
        new_splits[split_name] = split_data.select(selected_indices)

    return DatasetDict(new_splits)


subset_dataset = stratified_subset_dataset(dataset, n_samples_per_split=2000)  # DICA: tente modificar o parâmetro n_samples_per_split

In [25]:
print(len(subset_dataset['train']))
print(len(subset_dataset['test']))

2000
2000


*   ## **Para as próximas etapas use o laborátorio apresentado em aula**

# Tokenização

Nessa seção vocês vão aprender a **transformar frases em tokens** (unidades menores de texto, como palavras ou subpalavras), que é o formato que o modelo consegue entender.  

Lembre-se:  
- O modelo **DistilBERT** não trabalha diretamente com texto cru, mas sim com **IDs numéricos** que representam tokens.  
- O **tokenizer** faz esse processo automaticamente:  
  1. Divide a frase em tokens.  
  2. Converte os tokens para IDs.  
  3. Garante que todos os exemplos tenham o mesmo tamanho (com *padding* e *truncation*). Lembre-se de tentar diferentes tamanhos de tokens de entrada do modelo.  

In [26]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

seq_length = 128

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=seq_length)


tokenized_datasets = subset_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
mask_id = tokenizer.mask_token_id

print(f"Token ID for [CLS]: {cls_id}")
print(f"Token ID for [SEP]: {sep_id}")
print(f"Token ID for [MASK]: {mask_id}")

print(tokenized_datasets['train'][0])

Token ID for [CLS]: 101
Token ID for [SEP]: 102
Token ID for [MASK]: 103
{'label': 0, 'input_ids': [101, 1045, 4149, 2023, 2678, 2012, 24547, 22345, 1005, 1055, 1002, 1015, 8026, 1012, 1045, 2228, 1045, 2058, 1011, 3825, 999, 999, 999, 1999, 1996, 7675, 1010, 20252, 11320, 12333, 2072, 2081, 1037, 2146, 5164, 1997, 3822, 1011, 3446, 5691, 2005, 2235, 4835, 1006, 1999, 2023, 2553, 1010, 18847, 13113, 1011, 1011, 1996, 3924, 2040, 2081, 2087, 1997, 1996, 6812, 2854, 3337, 3152, 1007, 1012, 2096, 1996, 23277, 29574, 2791, 1997, 2087, 1997, 2122, 3152, 2515, 2025, 3921, 1996, 2504, 1997, 9643, 2791, 2010, 2197, 3152, 4719, 1006, 3968, 3536, 1000, 10002, 1000, 2107, 2004, 8959, 1997, 1996, 6071, 1998, 2933, 1023, 2013, 6058, 2686, 1007, 1010, 2027, 2024, 9690, 3532, 3152, 1998, 2323, 2022, 9511, 2011, 2035, 2021, 1996, 2087, 3280, 1011, 2524, 4599, 1012, 1026, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Adicionando Mascara aos Dados

Nessa etapa vocês vão aplicar a **máscara de linguagem (MLM)**, que é o coração do pré-treinamento do BERT/DistilBERT.  
A ideia é **esconder (mascarar) aleatoriamente tokens** da frase e pedir para o modelo tentar prever quais palavras estavam lá.  

**Dica:**
- Não é preciso reinventar tudo: a biblioteca `transformers` já possui funções que ajudam a criar essas máscaras.  
- Testem mascarar as frases considerando outras proporções além dos 15% originais do artigo.   

In [27]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.25):
    inputs = np.array(inputs)
    labels = np.copy(inputs)

    rand = np.random.rand(*inputs.shape)

    mask_arr = (rand < mlm_probability)

    special_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id]
    for special_id in special_tokens:
        mask_arr[inputs == special_id] = False

    inputs[mask_arr] = tokenizer.mask_token_id

    labels[~mask_arr] = 0

    return inputs, labels

sample_ids = tokenized_datasets['train'][0]['input_ids']
masked_inputs, labels = mask_tokens([sample_ids], tokenizer)

print("IDs Originais:\n", np.array(sample_ids))
print("\nIDs Mascarados:\n", masked_inputs[0])
print("\nLabels para Treinamento (0 significa 'ignorar'):\n", labels[0])

IDs Originais:
 [  101  1045  4149  2023  2678  2012 24547 22345  1005  1055  1002  1015
  8026  1012  1045  2228  1045  2058  1011  3825   999   999   999  1999
  1996  7675  1010 20252 11320 12333  2072  2081  1037  2146  5164  1997
  3822  1011  3446  5691  2005  2235  4835  1006  1999  2023  2553  1010
 18847 13113  1011  1011  1996  3924  2040  2081  2087  1997  1996  6812
  2854  3337  3152  1007  1012  2096  1996 23277 29574  2791  1997  2087
  1997  2122  3152  2515  2025  3921  1996  2504  1997  9643  2791  2010
  2197  3152  4719  1006  3968  3536  1000 10002  1000  2107  2004  8959
  1997  1996  6071  1998  2933  1023  2013  6058  2686  1007  1010  2027
  2024  9690  3532  3152  1998  2323  2022  9511  2011  2035  2021  1996
  2087  3280  1011  2524  4599  1012  1026   102]

IDs Mascarados:
 [  101  1045  4149  2023  2678  2012 24547 22345  1005   103  1002  1015
  8026  1012   103   103  1045   103   103   103   999   999   999  1999
  1996  7675   103 20252 11320 12333  20

# Adequando os Dados

Para treinar um modelo de linguagem de forma eficiente, precisamos **organizar os dados em lotes (batches)** e colocá-los em uma estrutura que o modelo consiga consumir durante o treinamento.  

**Dica:**
- Testem com diferentes tamanhos de `batch_size` e observem como isso afeta a memória e a velocidade.  
- Conferir os **shapes** dos tensores é sempre uma boa prática para evitar erros no treinamento.  

In [28]:
def tf_dataset(tokenized_inputs, tokenizer, batch_size=8, max_samples=None): #
    """Cria um tf.data.Dataset otimizado para o treinamento de MLM."""
    if max_samples:
        # Se um número máximo de amostras for especificado, seleciona um subconjunto
        labels_stratify = np.array(tokenized_inputs['label'])
        indices = np.arange(len(labels_stratify))
        selected_indices, _ = train_test_split(
            indices,
            train_size=max_samples,
            stratify=labels_stratify,
            random_state=42
        )
        tokenized_inputs = tokenized_inputs.select(selected_indices) #

    # Converte as listas de IDs para arrays NumPy
    input_ids = np.array(tokenized_inputs['input_ids'], dtype=np.int32) #
    attention_mask = np.array(tokenized_inputs['attention_mask'], dtype=np.int32) #

    # Aplica a função de mascaramento que criamos na seção anterior
    inputs, labels = mask_tokens(input_ids, tokenizer) #
    labels = labels.astype(np.int32) #

    # Cria o dataset TensorFlow a partir dos tensores
    dataset = tf.data.Dataset.from_tensor_slices(
        (
            {'input_ids': inputs, 'attention_mask': attention_mask}, # Features
            labels  # Labels
        )
    ) #

    # Embaralha os dados e os agrupa em lotes (batches)
    # DICA: Experimente diferentes tamanhos de batch_size.
    return dataset.shuffle(1000, seed=42).batch(batch_size) #

# Cria os datasets de treino e validação para a tarefa de MLM
# Usando 1000 amostras para um treinamento mais rápido
train_dataset = tf_dataset(tokenized_datasets['train'], tokenizer, batch_size=16, max_samples=1000) #
val_dataset = tf_dataset(tokenized_datasets['test'], tokenizer, batch_size=16, max_samples=1000) #

# Carregando dados de um modelo pré-treinado para continuar o pré-treinamento

Até aqui vocês aprenderam a preparar dados (tokenizar, mascarar e estruturar em batches).  
Agora, em vez de treinar um modelo do zero, vamos **aproveitar o conhecimento de um modelo já pré-treinado** e continuar o pré-treinamento com o nosso conjunto de dados (IMDB).  

Por que usar um modelo pré-treinado?
- Modelos como **DistilBERT** já foram treinados em enormes quantidades de texto.  
- Isso significa que eles **já entendem bastante da linguagem** (sintaxe, gramática, vocabulário, contexto).  
- Ao continuar o pré-treinamento com um novo corpus, você adapta o modelo para **captar melhor o estilo e o domínio** dos seus dados.  
- Lembre-se de experimentar diferentes quantidades de épocas e valores de learning-rate.  


In [30]:
from transformers import TFAutoModelForMaskedLM

try:
    model = TFAutoModelForMaskedLM.from_pretrained('distilbert-base-uncased', from_pt=True)
except TypeError:
    model = TFAutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')


optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

epochs = 3

model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForMaskedLM: ['vocab_projector.weight']
- This IS expected if you are initializing TFDistilBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Epoch 1/3


KeyboardInterrupt: 

# Testando as Predições

**Atenção use esses três exemplos para testar!!!**

In [ ]:
def predict_masked_token(text, model, tokenizer, top_k=5):
    inputs = tokenizer(text, return_tensors='tf', padding='max_length', truncation=True, max_length=seq_length)
    input_ids = inputs["input_ids"]

    logits = model(inputs).logits

    mask_position = tf.where(input_ids[0] == tokenizer.mask_token_id)[0, 0]

    mask_logits = logits[0, mask_position]

    mask_probs = tf.nn.softmax(mask_logits)

    top_k_ids = tf.argsort(mask_probs, direction="DESCENDING")[:top_k]
    top_k_probs = tf.gather(mask_probs, top_k_ids)

    print(f"Texto Original: {text}")
    print(f"Top {top_k} previsões para [MASK]:")
    for token_id, prob in zip(top_k_ids.numpy(), top_k_probs.numpy()):
        print(f"- {tokenizer.decode([token_id])} (Prob: {prob:.4f})")
    print("-" * 30)

text1 = "The movie was really [MASK]."
predict_masked_token(text1, model, tokenizer)

text2 = "The food at the restaurant was absolutely [MASK]."
predict_masked_token(text2, model, tokenizer)

text3 = "The weather today is very [MASK]."
predict_masked_token(text3, model, tokenizer)

Busque alguma frase de exemplo da base de dados de teste e coloque a máscara em lugares variados do texto.

In [ ]:
#Busque alguma frase no conjunto de teste
test_sentence = subset_dataset['test'][10]['text']
words = test_sentence.split()
if len(words) > 10:
    words[10] = "[MASK]"
    masked_test_text = " ".join(words)
    predict_masked_token(masked_test_text, model, tokenizer)

# Finetuning Downstream (Classificação)

Até agora o foco foi no **pré-treinamento com Masked Language Modeling (MLM)**, onde o objetivo era prever palavras mascaradas.  
Mas o poder do BERT/DistilBERT aparece de verdade quando usamos esse conhecimento adquirido em **tarefas downstream** (tarefas específicas), como classificação.  

**O que muda aqui?**
- No pré-treinamento, o modelo aprende sobre a **linguagem em geral**.  
- No fine-tuning, ajustamos o modelo para uma tarefa **específica**, por exemplo:  
  - **Classificação de sentimentos** (positivo/negativo em reviews do IMDB).  
  - **Classificação de tópicos**.  
  - **Detecção de spam**.  

**O que vocês precisam fazer aqui:**
- Utilizar um modelo pré-treinado (pode ser o seu ou não) (`DistilBERT`).  
- Adaptá-lo para classificação usando `TFDistilBertForSequenceClassification` (ou versão PyTorch).  
- Preparar os dados de entrada (frases e rótulos → `0` para negativo, `1` para positivo, por exemplo).  
- Treinar o modelo nos dados rotulados.
  
**Objetivo final:**  
Transformar um modelo genérico (pré-treinado em linguagem) em um modelo **especializado em classificar sentimentos no IMDB**.

**Dicas:**
- Ajustem **taxa de aprendizado** e **batch size**: esses hiperparâmetros têm forte impacto no desempenho.  
- Depois de treinar, calcule as métricas para verificar se o modelo capturou bem os padrões de classificação.
- Tente obter resultados melhores do que os do notebook exemplo da aula.  



In [ ]:
from transformers import TFDistilBertForSequenceClassification
from sklearn.metrics import classification_report

def tf_dataset_for_classification(tokenized_inputs, batch_size=16):
    """Cria um tf.data.Dataset para tarefas de classificação."""
    input_ids = np.array(tokenized_inputs['input_ids'], dtype=np.int32)
    attention_mask = np.array(tokenized_inputs['attention_mask'], dtype=np.int32)
    labels = np.array(tokenized_inputs['label'], dtype=np.int32)

    dataset = tf.data.Dataset.from_tensor_slices(
        (
            {'input_ids': input_ids, 'attention_mask': attention_mask},
            labels
        )
    )
    return dataset.shuffle(1000).batch(batch_size)

clf_train_dataset = tf_dataset_for_classification(tokenized_datasets['train'])
clf_test_dataset = tf_dataset_for_classification(tokenized_datasets['test'])


clf_model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)

clf_optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
clf_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
clf_metrics = ['accuracy']

clf_model.compile(optimizer=clf_optimizer, loss=clf_loss, metrics=clf_metrics)

clf_epochs = 3
clf_model.fit(clf_train_dataset, validation_data=clf_test_dataset, epochs=clf_epochs)

y_true = np.concatenate([y.numpy() for x, y in clf_test_dataset], axis=0)

predictions = clf_model.predict(clf_test_dataset)
y_pred = np.argmax(predictions.logits, axis=1) #

print("\n--- Relatório de Avaliação do Modelo de Classificação ---")
print(classification_report(y_true, y_pred, target_names=["NEGATIVE", "POSITIVE"]))